In [29]:
import pickle
import pandas as pd
from bs4 import BeautifulSoup

In [30]:
# Load the dictionary from the pickle file
with open('mydict_dict.pickle', 'rb') as file:
    mydict = pickle.load(file)

In [31]:
mustra = '''<!DOCTYPE html>
<html>
<head>
<style>
    table {
        width: 75%; 
        margin-left: auto; 
        margin-right: auto;
        font-family: Arial, Helvetica, sans-serif; 
        line-height: 1.5;
        border: 1px solid black; 
        border-collapse: collapse; 
        padding: 1rem;    
    }

    td {
        white-space: pre-wrap;
        border: 1px solid black;
        border-collapse: collapse;
        padding: 1rem;        
    }

    th {
        border: 1px solid black;
        border-collapse: collapse;
        padding: 1rem;          
    }
    
    .datum, .meniny {
        text-wrap: nowrap;
        text-align: center
    }
</style>
</head>
<body>
<table>
<th>dátum</th>
<th>deň</th>
<th>sviatok</th>
<th>poznámka</th>
</table>
</body>
</html>
'''

In [32]:
soup = BeautifulSoup(mustra , 'html.parser')

In [33]:
for key in mydict.keys():
    tr = soup.new_tag('tr')
    tr['id'] = key # id='2023-01-01'

    td1 = soup.new_tag('td')
    td1.string = key # '2023-01-01' dátum
    td1['class'] = "datum" # na túto class je napojené CSS, aby sa nezalamoval dátum string.

    # tu rowspan riešim
    velkost = len(mydict[key]) # length of nested dictionary
    if velkost>2: # zvyčajne obsahuje 2 polia: "meniny" a "0"; ak viac polí, v dni je viac možností (0,1,2).
        td1['rowspan']=str(velkost-1) # a musíme nastaviť rowspan, podľa počtu možností

    tr.append(td1)

    for variable in mydict[key].keys(): # prechádzame nested dictionary
        if variable == 'meniny': # pole "meniny"
            td2 = soup.new_tag('td')
            td2['class']='meniny' # na túto class je napojené CSS, aby sa nezalamoval m_string.

            # meniny string obohatený o ďalšie údaje o dni, this will be crucial
            meniny = mydict[key][variable]
            current_date = pd.to_datetime(key)
            m_string = str(current_date.day)+'. '+current_date.month_name('sk_SK').lower()+', '+current_date.day_name('sk_SK').lower()+'\n<i>'+meniny+'</i>'
            app = BeautifulSoup(m_string,'html.parser')
            td2.append(app)

            if velkost>2:
                td2['rowspan']=str(velkost-1)      

            tr.append(td2)

        else: # zvyšné polia (0,1,2)
            td2 = soup.new_tag('td')
            td2.string = mydict[key][variable][0] # v liturgii... nultý (prvý) údaj v liste
            tr.append(td2)

            td2 = soup.new_tag('td')
            td2.string = mydict[key][variable][1] # poznámka... prvý (druhý) údaj v liste
            tr.append(td2)

            if velkost>2: # ak je v dni je viac možností, musíme pridať ďalší riadok (row)
                soup.table.append(tr)
                tr = soup.new_tag('tr')

    soup.table.append(tr)

In [34]:
# soup

In [35]:
with open('plain_smaller.html', mode='w', encoding='utf8') as f:
    f.write(str(soup))

In [8]:
# done.